2021/01/01 ~ 2021/01/16 의 뉴스를 긁어와서\
- media : 언론사\
- scr : url\
- title : 제목\
- date : 날짜

In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import re


""" 네이버 랭킹 뉴스 긁어오기 """
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'}

d_list = []
start_data = 20230101
end_data = 20230113
for date_int in range(start_data, end_data):
    date = str(date_int)
    url = "https://news.naver.com/main/ranking/popularDay.nhn?date=" + date
    html = requests.get(url, headers=headers).text
    soup = BS(html, 'html.parser')
    ranking_total = soup.find_all(class_='rankingnews_box')

    for item in ranking_total:
        media = item.a.strong.text
        news = item.find_all(class_="list_content")
        for new in news:
            d = {}
            d['media'] = media
            d['src'] = "https://news.naver.com/" + new.a['href']
            d['title'] = new.a.text
            d['date'] = date
            d_list.append(d)
df = pd.DataFrame(d_list)

In [2]:
""" 필요 없는 문자 제거 """
def clean_text(row):
    text = row['title']
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("E-mail제거 : " , text , "\n")
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("URL 제거 : ", text , "\n")
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("한글 자음 모음 제거 : ", text , "\n")
    pattern = '<[^>]*>'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("태그 제거 : " , text , "\n")
    pattern = r'\([^)]*\)'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("괄호와 괄호안 글자 제거 :  " , text , "\n")
    pattern = '[^\w\s]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("특수기호 제거 : ", text , "\n" )
    pattern = '[^\w\s]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("필요없는 정보 제거 : ", text , "\n" )
    pattern = '["단독"]'
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '["속보"]'
    text = re.sub(pattern=pattern, repl='', string=text)
    # print("단독 속보 제거 : ", text , "\n" )
    text = text.strip()
    # print("양 끝 공백 제거 : ", text , "\n" )
    text = " ".join(text.split())
    # print("중간에 공백은 1개만 : ", text )
    return text


df['title_c'] = df.apply(clean_text, axis=1)

In [20]:
""" 키워드 추출 from title """
from konlpy.tag import Kkma
from konlpy.tag import Komoran


kkma = Kkma()
komoran = Komoran()

df['keyword'] = ''
for idx_line in range(len(df)):
    nouns_list = komoran.nouns(df['title_c'].loc[idx_line])
    nouns_list_c = [nouns for nouns in nouns_list if len(nouns) > 1]   # 한글자는 이상한게 많아서 2글자 이상
    a = set(nouns_list_c)
    df.loc[[idx_line], 'keyword'] = a
df = df[df['media'] != '코리아헤럴드']    # 코리아헤럴드는 영어 기사

network graph\
- network type : undirected weighted network
- node : 키워드
- edge : 동일 기사 제목에 등장한 키워드들은 관련이 있을 확률이 높으므로, 키워드들 간에 link 추거. 다른 기사에서 또 다시 link 추가될 시 weight 1 추가

In [33]:
""" Edge list 작성 """
from collections import Counter


edge_list = []
for keywords_dict in df['keyword']:
    keywords = list(keywords_dict)
    num_keyword = len(keywords)
    if num_keyword > 0:
        for i in range(num_keyword-1):
            for j in range(i+1, num_keyword):
                edge_list += [tuple(sorted([keywords[i], keywords[j]]))]    # node 간 위해 sorted 사용
edges = list(Counter(edge_list).items())



""" networkx Graph 작성 """
import networkx as nx


G = nx.Graph((x, y, {'weight': v}) for (x, y), v in edges)

In [30]:
""" Community 추출 """
#%pip install python-louvain
#%pip install networkx
#%pip install cdlib

from cdlib import algorithms
import networkx as nx
coms = algorithms.louvain(G, resolution=1., randomize=False)

nx.set_node_attributes(G, coms, "community")   # graph G에 community 속성 추가

In [34]:
""" Gephi file 작성 """
nx.write_gexf(G, '202301_keyword_community.gexf')